In [1]:
import decord
import json
import torch
from transformers import CLIPProcessor, CLIPModel
import numpy as np
from tqdm import tqdm

d:\Anaconda\envs\sklearn-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/train_data.json", "r") as f:
    train_data = json.load(f)
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/train_label.json", "r") as f:
    train_label = json.load(f)
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/val_data.json", "r") as f:
    val_data = json.load(f)
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/val_label.json", "r") as f:
    val_label = json.load(f)


In [2]:
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/test_data.json", "r") as f:
    test_data = json.load(f)

In [3]:
model = CLIPModel.from_pretrained("D:/allforwork/AI/nnsp/CLIP/clip-vit-base-patch32",low_cpu_mem_usage=True)
processor = CLIPProcessor.from_pretrained("D:/allforwork/AI/nnsp/CLIP/clip-vit-base-patch32")
model = model.to('cuda').eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
all_len = []
for q in test_data:
    video_path = f"D:/allforwork/AI/nnsp/NS-2025-10-data/video/{q['video']}"
    question_input = [f"Q: {q['question']}?\nA:{q[f'a{i}']}" for i in range(5)]
    
    vr = decord.VideoReader(video_path, ctx=decord.cpu(0))
    all_len.append(len(vr))


<built-in method mean of numpy.ndarray object at 0x0000015A085E47B0>
<built-in method max of numpy.ndarray object at 0x0000015A085E47B0>
<built-in method min of numpy.ndarray object at 0x0000015A085E47B0>


In [5]:
all_len = np.array(all_len)
print(all_len.mean())
print(all_len.max())
print(all_len.min())

1253.9272300469484
2697
300


In [4]:
results = []
print(torch.cuda.is_available())
num_frames = 1800

True


In [5]:
for q in tqdm(test_data):
    video_path = f"D:/allforwork/AI/nnsp/NS-2025-10-data/video/{q['video']}"
    question_input = [f"Q: {q['question']}?\nA:{q[f'a{i}']}" for i in range(5)]
    
    vr = decord.VideoReader(video_path, ctx=decord.cpu(0))
    frame_indices = np.linspace(0, len(vr)-1, num=min(num_frames, len(vr)), dtype=int)
    batch_frames = []
    for idx in frame_indices:
        frame = vr[idx].asnumpy()
        batch_frames.append(frame)
    
    
    inputs = processor(text=question_input,
                       images=batch_frames,
                       return_tensors="pt",
                       padding=True).to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image.cpu()
    probs = logits_per_image.softmax(dim=-1).mean(dim=0)
    results.append(probs.argmax().item())


100%|██████████| 426/426 [1:33:48<00:00, 13.21s/it]


In [6]:
results


[0,
 3,
 2,
 2,
 0,
 4,
 1,
 4,
 3,
 3,
 2,
 0,
 2,
 4,
 3,
 2,
 0,
 1,
 3,
 4,
 1,
 0,
 2,
 4,
 0,
 0,
 2,
 4,
 1,
 1,
 2,
 2,
 1,
 4,
 3,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 4,
 2,
 2,
 1,
 1,
 4,
 1,
 4,
 2,
 0,
 0,
 3,
 3,
 0,
 1,
 1,
 2,
 1,
 2,
 4,
 2,
 4,
 3,
 4,
 2,
 4,
 3,
 2,
 1,
 2,
 3,
 1,
 4,
 4,
 2,
 2,
 2,
 4,
 3,
 3,
 4,
 1,
 1,
 4,
 3,
 2,
 3,
 1,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 3,
 2,
 3,
 2,
 4,
 0,
 2,
 3,
 4,
 2,
 3,
 3,
 4,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 3,
 4,
 0,
 1,
 1,
 0,
 4,
 3,
 2,
 1,
 0,
 3,
 0,
 3,
 4,
 1,
 2,
 4,
 0,
 4,
 2,
 3,
 2,
 2,
 2,
 0,
 3,
 0,
 0,
 0,
 3,
 2,
 4,
 0,
 4,
 0,
 4,
 3,
 2,
 2,
 0,
 1,
 3,
 2,
 1,
 1,
 4,
 4,
 0,
 4,
 4,
 1,
 1,
 0,
 2,
 4,
 1,
 3,
 2,
 3,
 1,
 4,
 4,
 3,
 0,
 4,
 3,
 1,
 1,
 0,
 3,
 0,
 2,
 2,
 3,
 2,
 1,
 4,
 3,
 1,
 2,
 2,
 2,
 3,
 3,
 0,
 1,
 0,
 3,
 3,
 0,
 4,
 3,
 0,
 4,
 0,
 4,
 3,
 3,
 3,
 2,
 2,
 4,
 4,
 1,
 4,
 1,
 3,
 3,
 1,
 0,
 3,
 2,
 3,
 2,
 4,
 4,
 2,
 3,
 0,
 1,
 0,
 4,
 0,
 3,
 4,
 4,
 1,
 1,
 1,
 3,


In [7]:
ans = [str(i) for i in results]
ans[:5]

['0', '3', '2', '2', '0']

In [8]:
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/results.json", "w") as f:
    json.dump(ans, f, indent=4)